item-based sander en hugo

In [1]:
import json
from pandas import Series, Timestamp
import pandas as pd
from pandas import DataFrame
from numpy import nan
import sklearn.metrics.pairwise as pw
import numpy as np


In [2]:
def load_jsons(data_path, file):
    """ helper function to load '.json' files (they're not proper jsons) """
    file_path = data_path + file
    with open(file_path) as jsons:
        lines = [json.loads(json_line) for json_line in jsons]
    return pd.DataFrame(lines)

In [37]:
data_path = '../data/las vegas/'
review_file = 'review.json'
reviews = load_jsons(data_path, review_file)

display(reviews.head())

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33
2,RS_GTIT6836bCaPy637kNQ,nMeCE5-xsdleyxYuNZ_7rA,oxwGyA17NL6c5t1Etg5WgQ,3.0,1,0,1,It's a giant Best Buy with 66 registers. I do...,2012-02-29 21:52:43
3,kbtscdyz6lvrtGjD1quQTg,FIk4lQQu1eTe2EpzQ4xhBA,8mIrX_LrOnAqWsB5JrOojQ,4.0,0,0,0,"Like walking back in time, every Saturday morn...",2011-11-30 02:11:15
4,qlXw1JQ0UodW7qrmVgwCXw,bAhqAPoWaZYcyYi7bs024Q,LUN6swQYa4xJKaM_UEUOEw,4.0,0,0,0,Michael from Red Carpet VIP is amazing ! I rea...,2018-04-27 20:25:26


In [ ]:
def split_data(data, d = 0.75):
    np.random.seed(seed=5)
    mask_test = np.random.rand(data.shape[0]) < d
    return data[mask_test], data[~mask_test]

reviews_training, reviews_test = split_data(reviews, d=0.9)

In [ ]:
def pivot_ratings(df):
    return df.pivot_table(values='stars', columns='user_id', index='business_id')

def create_similarity_matrix_cosine(matrix):
    mc_matrix = matrix - matrix.mean(axis = 0)
    return pd.DataFrame(pw.cosine_similarity(mc_matrix.fillna(0)), index = matrix.index, columns = matrix.index)

In [ ]:
review_utitity_ratings = pivot_ratings(reviews_training)
display(review_utitity_ratings.head())

In [ ]:
review_similarity_ratings = create_similarity_matrix_cosine(review_utitity_ratings)
display(review_similarity_ratings.head())

In [ ]:
def predict_ratings(similarity, utility, to_predict):
    """Predicts the predicted rating for the input test data.
    
    Arguments:
    similarity -- a dataFrame that describes the similarity between items
    utility    -- a dataFrame that contains a rating for each user (columns) and each movie (rows). 
                  If a user did not rate an item the value np.nan is assumed. 
    to_predict -- A dataFrame containing at least the columns movieId and userId for which to do the predictions
    """
    # copy input (don't overwrite)
    ratings_test_c = to_predict.copy()
    # apply prediction to each row
    ratings_test_c['predicted rating'] = to_predict.apply(lambda row: predict_ids(similarity, utility, row['user_id'], row['business_id']), axis=1)
    return ratings_test_c
### Helper functions for predict_ratings_item_based ###

def predict_ids(similarity, utility, userId, itemId):
    # select right series from matrices and compute
    if userId in utility.columns and itemId in similarity.index:
        return predict_vectors(utility.loc[:,userId], similarity[itemId])
    return 0

def predict_vectors(user_ratings, similarities):
    # select only movies actually rated by user
    relevant_ratings = user_ratings.dropna()
    
    # select corresponding similairties
    similarities_s = similarities[relevant_ratings.index]
    
    # select neighborhood
    similarities_s = similarities_s[similarities_s > 0.0]
    relevant_ratings = relevant_ratings[similarities_s.index]
    
    # if there's nothing left return a prediction of 0
    norm = similarities_s.sum()
    if(norm == 0):
        return 0
    
    # compute a weighted average (i.e. neighborhood is all) 
    return np.dot(relevant_ratings, similarities_s)/norm

In [36]:
review_predicted_item_based = predict_ratings(review_similarity_ratings, review_utitity_ratings, reviews_test[['user_id', 'business_id', 'stars']])
display(review_predicted_item_based)

,user_id,business_id,stars,predicted rating
3,dUOg3fS3RTYDZUpu0CqqiA,729grSa1Wsn-hfv7D5uOxg,5.0,0.0
27,1kcOj0Ch394fLiUaC_fD9g,Q_0eGl-aElqHKukHvmLdwA,5.0,0.0
28,LIHlbBXL5LsOyNpQVAU3lg,Eu_zPTrNVAXkpdSxf7CJ2w,5.0,0.0
39,sNtu3HsJ7_2nXRBz-3_dIA,Eu_zPTrNVAXkpdSxf7CJ2w,5.0,0.0
43,xBK-ElArbwMaonFFAI4NAA,Yjf0i2J9q52dYIT8UVGT3g,5.0,0.0
46,91_o14HnvQlCqe8XSHadxQ,729grSa1Wsn-hfv7D5uOxg,5.0,0.0
56,6Ki3bAL0wx9ymbdJqbSWMA,729grSa1Wsn-hfv7D5uOxg,5.0,0.0
62,REJ4ZVhUKUsyxBAveg70zA,729grSa1Wsn-hfv7D5uOxg,3.0,0.0
63,3xQ9B-tv3D_fCnKRwt0c2w,729grSa1Wsn-hfv7D5uOxg,5.0,5.0
79,ANOuJtkXx3jkRcqDQ_80Tg,729grSa1Wsn-hfv7D5uOxg,5.0,0.0
